# 第7讲：视觉里程计1


## 7.1 特征点法

视觉SLAM分为视觉前端和优化后端。前端也称为视觉里程计（VO），根据相邻图像信息，粗略估计相机运动，为后端提供初始值。

VO的实现按是否需要提取特征，分为特征点法前端及不提特征的直接法前端。基于特征点法的前端运行稳定，对光照、动态物体不敏感，是比较成熟的解决方。


### 7.1.1 特征点

VO主要问题是根据图像来估计相机运动：

1. 从图像中选取代表性点，要求当相机视角发生少许变化时，这些点保持不变；

2. 根据这些点求解相机位姿估计问题及这些点的定位问题。

在经典SLAM模型中，这些代表性点称为**路标**。视觉SLAM中，路标指**图像特征（features）**。*特征是图像信息的一种数字表达形式*。视觉里程计要求*在相机运动前后，特征点保持稳定*，即当场景、相机视角发生少许改变时，仍能从图像中判断哪些像素属于同一位置。数字图像在计算机中以灰度值矩阵方式存储，因此图像像素是一种最简单的特征，但其易受光照、形变、物体材质影响，在不同图像之间变化非常大，不够稳定。

**特征点**：图像中*特别的地方*，角点、边缘和区块均可视为图像的代表性地方。其中，角点最容易识别；其次为边缘（图像局部沿着边缘相似）；区块最难识别。图像中的角点、边缘相比像素区块更“特别”，在不同图像之间辨识度更高。

<img src="./img/fig_7_1.png" width="450" />

在大多数应用中，单纯角点无法满足需求。例如，从远处看是角点的地方，当相机移近后，可能显示为非角点；或者当相机旋转后，角点外观会发生变化，也无法辨认出同一角点。

局部图像特征性质：

1. 可重复性（repeatability）：相同“区域”可在不同图像中被辩识；

2. 可区别性（distinctiveness）：不同“区域”的表达不同；

3. 高效率（efficiency）：同一图像特征点数量远小于像素数量；

4. 本地性（locality）：特征仅与图像局部区域相关。

特征点由关键点（Key-point）和描述子（Des


常用局部图像特征：`SIFT`、`SURF`、`ORB`